<a href="https://colab.research.google.com/github/urmzd/rabbit-classifier/blob/main/src/rc_e1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!mkdir -p /content
!curl https://raw.githubusercontent.com/urmzd/rabbit-classifier/main/resources/data.csv --output /content/data.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9459  100  9459    0     0   130k      0 --:--:-- --:--:-- --:--:--  130k


In [20]:
import pandas as pd
import requests
import os
import re
import pathlib

DATA_PATH="/content/data.csv"
IMAGE_PATH="/content/images"

df = pd.read_csv(DATA_PATH, usecols=range(2))

def init_image_directory(image_path=IMAGE_PATH) -> None:
  os.mkdir(image_path)

def download_image_to_location(link: str) -> pathlib.Path:
  file_name_regex = re.compile(r".*\/(.*\.(png|jpeg|jpg)).*", flags=re.IGNORECASE)
  regex_matches = file_name_regex.match(link)

  if not regex_matches:
    raise Exception(f"Failed to match file_name for link {link}")


  file_name = regex_matches.group(1)
  image_request_headers={
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
  }

  image = requests.get(link, headers=image_request_headers)
  print(image.status_code)

  content_path = pathlib.Path(IMAGE_PATH)
  content_path.mkdir(parents=True, exist_ok=True)

  file_path = content_path / file_name

  with open(file_path, "wb") as handle:
    handle.write(image.content)

  return file_path

file_path = download_image_to_location(df.iloc[2,1])

200


In [21]:
import cv2
from typing import NewType, Optional
import numpy as np
from google.colab.patches import cv2_imshow

Image = NewType('Image', np.array)

def read_and_show_image(file_path: pathlib.Path) -> Optional[Image]:
  img = cv2.imread(str(file_path))
  if img is None:
    return None

  #cv2_imshow(img)
  #return img

for _, row in df.iterrows():
  label, link = row
  try:
    path = download_image_to_location(link)  
  except:
    continue
  
  if path:
    read_and_show_image(path)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
